## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and sign up for a free account, and create your API keys.

As student Ron pointed out (thank you Ron!) there are actually 2 tokens to create in Pushover:  
1. The User token which you get from the home page of Pushover
2. The Application token which you get by going to https://pushover.net/apps/build and creating an app   

(This is so you could choose to organize your push notifications into different apps in the future.)


Add to your `.env` file:
```
PUSHOVER_USER=put_your_user_token_here
PUSHOVER_TOKEN=put_the_application_level_token_here
```

And install the Pushover app on your phone.

In [1]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [3]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [5]:
push("HEY Lee!!")

Push: HEY Lee!!


In [5]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [28]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [30]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [31]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [32]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [33]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [34]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [35]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [37]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [38]:
linkedin_reader = PdfReader("me/linkedin.pdf")
ibm_resume_reader = PdfReader("me/ibm_resume.pdf")
linkedin = ""
for page in linkedin_reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

ibmResume = ""
for page in ibm_resume_reader.pages:
    text = page.extract_text()
    if text:
        ibmResume += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Lee McCormick"

In [39]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n## IBM Resume:\n{ibmResume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [41]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [42]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool called: record_user_details
Push: Recording interest from Name not provided with email mauy0inter@gmail.com and notes not provided
Tool called: record_unknown_question
Push: Recording What is Lee McCormick's favorite movie? asked that I couldn't answer


## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces. Thank you student Robert M for improving these instructions.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you!  
Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions.
3. Take this token and add it to your .env file: `HF_TOKEN=hf_xxx` and see note below if this token doesn't seem to get picked up during deployment  
4. From the 1_foundations folder, enter: `uv run gradio deploy` and if for some reason this still wants you to enter your HF token, then interrupt it with ctrl+c and run this instead: `uv run dotenv -f ../.env run -- uv run gradio deploy` which forces your keys to all be set as environment variables   
5. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

#### Extra note about the HuggingFace token

A couple of students have mentioned the HuggingFace doesn't detect their token, even though it's in the .env file. Here are things to try:   
1. Restart Cursor   
2. Rerun load_dotenv(override=True) and use a new terminal (the + button on the top right of the Terminal)   
3. In the Terminal, run this before the gradio deploy: `$env:HF_TOKEN = "hf_XXXX"`  
Thank you James and Martins for these tips.  

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets, delete the space, etc.

#### And now you should be deployed!

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

In [15]:
# Exercise Code By Lee McCormick
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
from pydantic import BaseModel
import gradio as gr

load_dotenv(override=True)

def push(text):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )

def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push(f"Recording {question}")
    return {"recorded": "ok"}

record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

# Evaluation class
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

# Me class
class Me:
    def __init__(self):
        self.openai = OpenAI()
        self.gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"),  base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
        self.name = "Lee McCormick"

        # Read the files
        linkedin_reader = PdfReader("me/linkedin.pdf")
        ibm_resume_reader = PdfReader("me/ibm_resume.pdf")
        ccc_transcript_reader = PdfReader("me/ccc_transcript.pdf")

        # Initialize the variables
        self.linkedin = ""
        self.ibm_resume = ""
        self.ccc_transcript = ""

        # Extract the text from the files
        for page in linkedin_reader.pages:
            text = page.extract_text()
            if text:
                self.linkedin += text
        print(f"✅ self.linkedin --> {self.linkedin}")

        for page in ibm_resume_reader.pages:
            text = page.extract_text()
            if text:
                self.ibm_resume += text
        print(f"✅ self.ibm_resume --> {self.ibm_resume}")

        for page in ccc_transcript_reader.pages:
            text = page.extract_text()
            if text:
                self.ccc_transcript += text
        print(f"✅ self.ccc_transcript --> {self.ccc_transcript}")

        with open("me/summary.txt", "r", encoding="utf-8") as f:
            self.summary = f.read()
        print(f"✅ self.summary --> {self.summary}")    

    def handle_tool_call(self, tool_calls):
        results = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f"Tool called: {tool_name}", flush=True)
            tool = globals().get(tool_name)
            result = tool(**arguments) if tool else {}
            results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
        return results
    
    def system_prompt(self):
        system_prompt = f"You are acting as {self.name}. You are answering questions on {self.name}'s website, \
particularly questions related to {self.name}'s career, background, transcript, skills and experience. \
Your responsibility is to represent {self.name} for interactions on the website as faithfully as possible. \
You are given a summary of {self.name}'s background, LinkedIn profile, IBM resume, CCC transcript which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career, then ask to getting in touch via email for specific answer for the unknow question.   \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "
        system_prompt += f"\n\n## Summary:\n{self.summary}\n\n## LinkedIn Profile:\n{self.linkedin}\n\n## IBM Resume:\n{self.ibm_resume} \n\n## CCC Transcript:\n{self.ccc_transcript}\n\n"
        system_prompt += f"With this context, please chat with the user, always staying in character as {self.name}."
        return system_prompt
    
    def evaluator_system_prompt(self):
        evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {self.name} and is representing {self.name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {self.name} in the form of their summary and LinkedIn details. Here's the information:"
        evaluator_system_prompt += f"\n\n## Summary:\n{self.summary}\n\n## LinkedIn Profile:\n{self.linkedin}\n\n## IBM Resume:\n{self.ibm_resume} \n\n## CCC Transcript:\n{self.ccc_transcript}\n\n"
        evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."
        return evaluator_system_prompt
    
    def evaluator_user_prompt(self, reply, message, history):
        user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
        user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
        user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
        user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
        return user_prompt
    
    def evaluates(self, reply, message, history) -> Evaluation:
        messages = [{"role": "system", "content": self.evaluator_system_prompt()}] + [{"role": "user", "content": self.evaluator_user_prompt(reply, message, history)}]
        response = self.gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
        return response.choices[0].message.parsed
    
    def rerun(self, reply, message, history, feedback):
        updated_system_prompt = self.system_prompt() + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
        updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
        updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
        messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
        response = self.openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
        return response.choices[0].message.content

    def chat(self, message, history):
        messages = [{"role": "system", "content": self.system_prompt()}] + history + [{"role": "user", "content": message}]
        done = False

        while not done:
            response = self.openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
            if response.choices[0].finish_reason=="tool_calls":
                message = response.choices[0].message
                tool_calls = message.tool_calls
                results = self.handle_tool_call(tool_calls)
                messages.append(message)
                messages.extend(results)
            else:
                done = True
                reply = response.choices[0].message.content
                evaluation = self.evaluates(reply=reply, message=message, history=history)
    
                if evaluation.is_acceptable:
                    print("✅ Passed evaluation - returning reply")
                else:
                    print("❌ Failed evaluation - retrying")
                    print(evaluation.feedback)
                    reply = self.rerun(reply, message, history, evaluation.feedback)       
        return reply
    
# Run the code with interface using gradio.
me = Me()
gr.ChatInterface(me.chat, type="messages").launch()

Ignoring wrong pointing object 138 0 (offset 0)


✅ self.linkedin -->    
Contact
Norman, OK
312 549 1199 (Mobile)
leemccormick.developer@gmai
l.com
www.linkedin.com/in/
iosdeveloperleemccormick
(LinkedIn)
github.com/leemccormick
(Portfolio)
Top Skills
React.js
Mobile Applications
SwiftUI
Languages
English (Professional Working)
Thai (Native or Bilingual)
Certifications
IOS13 & Swift5 - The complete iOS
App Development Bootcamp
Basic Certificate Accounting
Jetpack Compose Crash course for
Android with Kotlin
[NEW] Spring Boot 3, Spring 6 &
Hibernate for Beginners 
The Complete Android N Developer
Course
Lee McCormick
Mobile Engineer | iOS Development | Swift | SwiftUI | Android
Development | React Native
Norman, Oklahoma, United States
Summary
I am a Mobile Engineer who specializes in creating applications
for Apple's iOS operating system. I love learning the necessary
programming languages, gaining hands-on experience through
projects, working with a team to deliver quality products and staying
up-to-date with the latest developments

✅ Passed evaluation - returning reply
Tool called: record_unknown_question
✅ Passed evaluation - returning reply
Tool called: record_user_details
✅ Passed evaluation - returning reply
✅ Passed evaluation - returning reply


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>

In [17]:
# Refactored Code by Claude AI
# Personal Assistant Bot by Lee McCormick
# A Gradio-based chatbot that represents Lee on his website with quality control

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
from pydantic import BaseModel
import gradio as gr
from typing import List, Dict, Any
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

load_dotenv(override=True)

# Notification system
def push_notification(text: str) -> None:
    """Send push notification via Pushover"""
    try:
        requests.post(
            "https://api.pushover.net/1/messages.json",
            data={
                "token": os.getenv("PUSHOVER_TOKEN"),
                "user": os.getenv("PUSHOVER_USER"),
                "message": text,
            },
            timeout=10
        )
    except Exception as e:
        logger.error(f"Failed to send notification: {e}")

# Tool functions
def record_user_details(email: str, name: str = "Name not provided", notes: str = "not provided") -> Dict[str, str]:
    """Record user contact details"""
    push_notification(f"New contact: {name} ({email}) - {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question: str) -> Dict[str, str]:
    """Record questions that couldn't be answered"""
    push_notification(f"Unknown question: {question}")
    return {"recorded": "ok"}

# Tool definitions for OpenAI
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "record_user_details",
            "description": "Record user contact information when they want to get in touch",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string", "description": "User's email address"},
                    "name": {"type": "string", "description": "User's name if provided"},
                    "notes": {"type": "string", "description": "Additional context about the conversation"}
                },
                "required": ["email"]
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "record_unknown_question",
            "description": "Record questions that couldn't be answered",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "The unanswered question"}
                },
                "required": ["question"]
            }
        }
    }
]

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

class PersonalAssistant:
    """AI assistant representing Lee McCormick on her website"""
    
    def __init__(self, data_dir: str = "me"):
        self.name = "Lee McCormick"
        self.data_dir = data_dir
        self.max_retries = 2
        
        # Initialize AI clients
        self._init_ai_clients()
        
        # Load personal data
        self._load_personal_data()
    
    def _init_ai_clients(self) -> None:
        """Initialize OpenAI and Gemini clients"""
        try:
            self.openai = OpenAI()
            self.gemini = OpenAI(
                api_key=os.getenv("GOOGLE_API_KEY"),
                base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
            )
        except Exception as e:
            logger.error(f"Failed to initialize AI clients: {e}")
            raise
    
    def _load_personal_data(self) -> None:
        """Load all personal documents and data"""
        try:
            # Load PDFs
            self.linkedin = self._extract_pdf_text("linkedin.pdf")
            self.ibm_resume = self._extract_pdf_text("ibm_resume.pdf") 
            self.ccc_transcript = self._extract_pdf_text("ccc_transcript.pdf")
            
            # Load summary
            with open(f"{self.data_dir}/summary.txt", "r", encoding="utf-8") as f:
                self.summary = f.read()
                
            logger.info("✅ All personal data loaded successfully")
            
        except Exception as e:
            logger.error(f"Failed to load personal data: {e}")
            raise
    
    def _extract_pdf_text(self, filename: str) -> str:
        """Extract text from PDF file"""
        filepath = f"{self.data_dir}/{filename}"
        if not os.path.exists(filepath):
            logger.warning(f"File not found: {filepath}")
            return ""
            
        try:
            reader = PdfReader(filepath)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
            return text
        except Exception as e:
            logger.error(f"Failed to extract text from {filename}: {e}")
            return ""
    
    def _handle_tool_calls(self, tool_calls) -> List[Dict[str, Any]]:
        """Execute tool calls and return results"""
        results = []
        for tool_call in tool_calls:
            try:
                tool_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                logger.info(f"Executing tool: {tool_name}")
                
                # Get function from globals and execute
                tool_func = globals().get(tool_name)
                if tool_func:
                    result = tool_func(**arguments)
                else:
                    result = {"error": f"Tool {tool_name} not found"}
                    
                results.append({
                    "role": "tool",
                    "content": json.dumps(result),
                    "tool_call_id": tool_call.id
                })
            except Exception as e:
                logger.error(f"Tool execution failed: {e}")
                results.append({
                    "role": "tool", 
                    "content": json.dumps({"error": str(e)}),
                    "tool_call_id": tool_call.id
                })
        return results
    
    def _get_system_prompt(self, feedback: str = None) -> str:
        """Generate system prompt with optional feedback"""
        base_prompt = f"""You are {self.name}, answering questions on your website about your career, background, and experience.
Be professional and engaging, as if talking to potential clients or employers. Use the provided documents to answer questions accurately.
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career, then ask to getting in touch via email for specific answer for the unknow question.   \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "
## Your Background:
Summary: {self.summary}
LinkedIn: {self.linkedin}
Resume: {self.ibm_resume}
Transcript: {self.ccc_transcript}
Stay in character as {self.name} and be helpful and professional."""
        if feedback:
            base_prompt += f"\n\n## Previous Response Rejected\nFeedback: {feedback}\nPlease improve your response."
        return base_prompt
    
    def _evaluate_response(self, reply: str, user_message: str, history: List[Dict]) -> Evaluation:
        """Evaluate response quality using Gemini"""
        try:
            evaluator_prompt = f"""Evaluate if this response is acceptable for {self.name}'s website assistant.
The assistant should be professional, engaging, and accurate based on the provided background information.
Background: {self.summary}
Conversation History: {history}
User Message: {user_message}  
Assistant Reply: {reply}
Is this response acceptable quality? Provide feedback."""
            messages = [
                {"role": "system", "content": evaluator_prompt},
                {"role": "user", "content": "Please evaluate this response."}
            ]
            
            response = self.gemini.beta.chat.completions.parse(
                model="gemini-2.0-flash",
                messages=messages,
                response_format=Evaluation
            )
            return response.choices[0].message.parsed
            
        except Exception as e:
            logger.error(f"Evaluation failed: {e}")
            # Default to acceptable if evaluation fails
            return Evaluation(is_acceptable=True, feedback="Evaluation service unavailable")
    
    def chat(self, message: str, history: List[Dict]) -> str:
        """Main chat function with quality control"""
        messages = [{"role": "system", "content": self._get_system_prompt()}] + history + [{"role": "user", "content": message}]
        
        for attempt in range(self.max_retries + 1):
            try:
                # Generate response with potential tool calls
                while True:
                    response = self.openai.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=messages,
                        tools=TOOLS
                    )
                    
                    if response.choices[0].finish_reason == "tool_calls":
                        # Handle tool calls
                        response_message = response.choices[0].message
                        tool_results = self._handle_tool_calls(response_message.tool_calls)
                        messages.append(response_message)
                        messages.extend(tool_results)
                    else:
                        # Got final response
                        reply = response.choices[0].message.content
                        break
                
                # Evaluate response quality
                evaluation = self._evaluate_response(reply, message, history)
                
                if evaluation.is_acceptable:
                    logger.info("✅ Response approved")
                    return reply
                else:
                    logger.warning(f"❌ Response rejected (attempt {attempt + 1}): {evaluation.feedback}")
                    if attempt < self.max_retries:
                        # Retry with feedback
                        messages = [{"role": "system", "content": self._get_system_prompt(evaluation.feedback)}] + history + [{"role": "user", "content": message}]
                    else:
                        # Max retries reached, return anyway
                        logger.warning("Max retries reached, returning response")
                        return reply
                        
            except Exception as e:
                logger.error(f"Chat error (attempt {attempt + 1}): {e}")
                if attempt == self.max_retries:
                    return "I apologize, but I'm experiencing technical difficulties. Please try again later."
        
        return reply

# Initialize and launch
if __name__ == "__main__":
    try:
        assistant = PersonalAssistant()
        interface = gr.ChatInterface(
            assistant.chat,
            type="messages",
            title=f"Chat with {assistant.name}",
            description="Ask me about my background, experience, and projects!"
        )
        interface.launch()
    except Exception as e:
        logger.error(f"Failed to start application: {e}")
        print(f"Error: {e}")

INFO:__main__:✅ All personal data loaded successfully
INFO:httpx:HTTP Request: GET http://127.0.0.1:7876/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7876/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Response approved
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Executing tool: record_unknown_question
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:✅ Response approved
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Executing tool: record_user_details
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativel